In [1]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers import Dense, Dropout 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("../input/training/training.csv")
test_df = pd.read_csv("../input/test/test.csv")

In [3]:
train_X = StandardScaler().fit(train_df.iloc[:,:-2].values).transform(train_df.iloc[:,:-2].values)
train_Y = train_df.iloc[:,-1:].values

In [4]:
encode = LabelEncoder()
encode.fit(train_Y)
train_Y = encode.transform(train_Y)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
train_Y

array([1, 0, 0, ..., 1, 0, 0])

In [6]:
train_df.head()

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,1.582,1.396,0.2,32.638,1.017,0.381,51.626,2.273,-2.414,16.824,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,125.157,0.879,1.414,-999.0,42.014,2.039,-3.011,36.918,0.501,0.103,44.704,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,197.814,3.776,1.414,-999.0,32.154,-0.705,-2.093,121.409,-0.953,1.052,54.283,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,75.968,2.354,-1.285,-999.0,22.647,-1.655,0.010,53.321,-0.522,-3.100,31.082,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,57.983,1.056,-1.385,-999.0,28.209,-2.197,-2.231,29.774,0.798,1.569,2.723,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


In [7]:
train_X

array([[-1.73204388,  0.46141372,  0.06833197, ...,  1.5668    ,
         1.55858439,  0.4125105 ],
       [-1.73203002,  0.51670419,  0.55250482, ..., -0.63936657,
        -0.63936694, -0.27381996],
       [-1.73201617, -2.33785898,  3.19515553, ..., -0.63936657,
        -0.63936694, -0.29396985],
       ...,
       [ 1.73201617,  0.38016991,  0.31931645, ..., -0.63936657,
        -0.63936694, -0.31701723],
       [ 1.73203002,  0.35431502, -0.84532397, ..., -0.63936657,
        -0.63936694, -0.74543941],
       [ 1.73204388, -2.33785898,  0.66533608, ..., -0.63936657,
        -0.63936694, -0.74543941]])

In [8]:
x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, test_size = 0.3)

In [9]:
model = Sequential()
model.add(Dense(31, input_shape=(31,), kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(31, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(31, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
opt = SGD(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt,metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 31)                992       
_________________________________________________________________
dropout_1 (Dropout)          (None, 31)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 31)                992       
_________________________________________________________________
dropout_2 (Dropout)          (None, 31)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 31)                992       
_________________________________________________________________
dropout_3 (Dropout)          (None, 31)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 32        
Total para

In [10]:
results = model.fit(
 train_X, train_Y,
 epochs= 10,
 batch_size = 64
)

Epoch 1/10
250000/250000 [==============================] - 13s 50us/step - loss: 0.6451 - acc: 0.6570
Epoch 2/10
250000/250000 [==============================] - 11s 45us/step - loss: 0.6232 - acc: 0.6573
Epoch 3/10
250000/250000 [==============================] - 11s 45us/step - loss: 0.5395 - acc: 0.7182
Epoch 4/10
250000/250000 [==============================] - 11s 45us/step - loss: 0.4932 - acc: 0.7813
Epoch 5/10
250000/250000 [==============================] - 11s 45us/step - loss: 0.4796 - acc: 0.7896
Epoch 6/10
250000/250000 [==============================] - 12s 49us/step - loss: 0.4700 - acc: 0.7955
Epoch 7/10
250000/250000 [==============================] - 12s 46us/step - loss: 0.4641 - acc: 0.7985
Epoch 8/10
250000/250000 [==============================] - 11s 45us/step - loss: 0.4582 - acc: 0.8018
Epoch 9/10
250000/250000 [==============================] - 11s 45us/step - loss: 0.4524 - acc: 0.8043
Epoch 10/10
250000/250000 [==============================] - 11s 45us/ste

In [11]:
test = StandardScaler().fit(test_df.values).transform(test_df.values)
prediction = model.predict_classes(test)

In [12]:
prediction

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [13]:
final = pd.DataFrame(columns=['EventId', 'Class'])
final['EventId'] = test_df['EventId']
final['Class'] = prediction

replacement = {
    1 : 's',
    0 : 'b'
}

final = final.replace({'Class' : replacement})

In [14]:
final.to_csv("submission.csv", header=True, sep="\t", encoding='utf-8', index=False)